In [ ]:
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from helper_functions import elementlines, nearestlines

In [ ]:
%matplotlib qt

### Variables that you need to change

- path
    - path to the data
- file
    - file name
- elements
    - list of elements to be analyzed
- lines_of_interest
    - lines of interest, will be included in the output
- info_on_all_lines
    - boolean, if true, the output will contain all lines, not only the lines of interest
- zero_peak
    - boolean, if true, the zero peak will be sliced out
- zero_peak_end_index
    - index of the last element of the zero peak, which will be sliced out
- k_to_l_ratio_elements
    - elements where the k to l ratio will be calculated, using Ka and La
- fwtm_to_fwhm_lines
    - lines of interest for the fwtm to fwhm ratio, should not be overlapping with other peaks
    - the fwhm and fwtm here is taken from the raw spectrum, since the fitted gaussians have perfect shapes
    - then if a overlapping peak is used, the ratio will be skewed

In [ ]:
##### SU9000 #####
# path = 'data/Skomedal_2022-03-23'
# file = 'Spectrum 03.emsa'
# elements = ['Al', 'Au', 'C', 'Cu', 'F', 'Fe', 'Mg', 'Mo', 'Ni', 'O', 'Si', 'Sn']  # from Mari, seems ok
# lines_of_interest = ['Al_Ka', 'C_Ka', 'Cu_Ka', 'Mo_La', 'Ni_Ka', 'Ni_La', 'O_Ka', 'Si_Ka']

##### GaAs APREO #####
path = 'data/Mæhlum_2022-09-06_EDS-SEM-APREO'
file = 'GaAs_30kV.emsa'
elements = ['Ga', 'As', 'O', 'C', 'Si']
lines_of_interest = ['Ga_Ka', 'As_Ka', 'O_Ka', 'C_Ka', 'Si_Ka', 'Ga_La', 'As_La', 'Ga_Kb', 'As_Kb', 'Ga_Ll']
info_on_all_lines = False
k_to_l_ratio_elements = ['As', 'Ga']
fwtm_to_fwhm_lines = ['Ga_Ka']

zero_peak = True 
zero_peak_end_index = 33

## The output parameters that will be calculated are:

...

In [ ]:
# output parameters
scale = np.nan  # float
offset = np.nan  # float 
energy_resolution = np.nan  # float
total_counts = np.nan  # float
background_counts = np.nan  # float
lines_info = dict() # dict of dicts as {line_name1: {true_energy: ..., calib_energy: ..., area: ..., max_counts: ..., sigma: ..., fwhm: ...}, line_name2: {...}}
k_to_l_ratios = dict()  # dict as {element1: k_to_l_ratio1, element2: k_to_l_ratio2, ...}
fwtm_to_fwhm = dict()  # dict as {line1: fwtm_to_fwhm1, line2: fwtm_to_fwhm2, ...}

## Loading the data and setting some parameters

In [ ]:
s = hs.load(path + '/' + file, signal_type='EDS_TEM')  # have to pretent it's a TEM signal
s.add_elements(elements)

# x_channels = np.arange(0, len(s.axes_manager[0].axis), 1)
if zero_peak:
    s = s.isig[zero_peak_end_index:]
    # x_channels = x_channels[zero_peak_end_index:]

Vacc = s.metadata.Acquisition_instrument.TEM.beam_energy
x_max = s.axes_manager[0].high_value  # highest x-axis value in keV, used in Duane-Hunt
x = s.axes_manager[0].axis  # x-axis in keV

In [ ]:
plt.rcParams['font.size'] = '16'
plt.rcParams["figure.figsize"] = (15,8)
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['font.family'] = 'monospace'
s.plot(xray_lines=True)

In [ ]:
# these are temporary arrays used to show the effect of the calibrations
# temporary because thay are not used in the final output

scale_list = [s.axes_manager[0].scale]
offset_list = [s.axes_manager[0].offset]
energy_res_list = [s.metadata.Acquisition_instrument.TEM.Detector.EDS.energy_resolution_MnKa]

In [ ]:
plt.close('all')

In [ ]:
m = s.create_model(auto_background=False)
m.plot()

In [ ]:
# fitting the data

m.add_polynomial_background(order=6) # use order 12? Or 8?
m.fit_background()
m.fit(bounded=True)
m.plot(True)

## Calibration (scale, offset and resolution)

In [ ]:
def calibrate_axis(rounds=2):
    # calibration of the energy axis, i.e. scale and offset
    print('Calibrating energy axis (with many elements it can take multiple minutes)')
           
    for i in range(rounds):
        print(f'Calibrating offset and scale, round {i+1} of {rounds}')
        m.calibrate_energy_axis(calibrate='offset')
        offset_list.append(s.axes_manager[0].offset)
        m.calibrate_energy_axis(calibrate='scale')
        scale_list.append(s.axes_manager[0].scale)

    print(f'Scale: {scale_list[-1]:.6f} eV/px \nOffset: {offset_list[-1]:.6f} keV')
    scale = scale_list[-1]
    offset = offset_list[-1]


def calibrate_resolution(rounds=1):
    # calibration of the energy resolution
    print('Calibrating energy resolution')
    for i in range(rounds):
        print(f'Calibrating energy resolution, round {i+1} of {rounds}')
        m.calibrate_energy_axis(calibrate='resolution')
        energy_res_list.append(s.metadata.Acquisition_instrument.TEM.Detector.EDS.energy_resolution_MnKa)

    print(f'Calibrated energy resolution: {energy_res_list[-1]:.3f} eV')
    energy_resolution = energy_res_list[-1]


def print_calibration_info():
    # make pretty print of calibration info
    infos = [' ', 'Scale [eV/channel]', 'Offset [keV]', 'E-res [eV]']
    row1 = ['Current', f'{scale_list[-1]:.6f}', f'{offset_list[-1]:.6f}', f'{energy_res_list[-1]:.3f}']
    row2 = ['Original', f'{scale_list[0]:.6f}', f'{offset_list[0]:.6f}', f'{energy_res_list[0]:.3f}']
    row3 = ['Δ original', f'{(scale_list[-1] - scale_list[0])/scale_list[-2]*100:.3f} %', 
    f'{(offset_list[-1] - offset_list[0])/offset_list[-2]*100:.3f} %', f'{(energy_res_list[-1] - energy_res_list[0])/energy_res_list[0]*100:.3f} %']
    row4 = ['Δ last step', f'{(scale_list[-1] - scale_list[-2])/scale_list[-2]*100:.3f} %', 
    f'{(offset_list[-1] - offset_list[-2])/offset_list[-2]*100:.3f} %', f'{(energy_res_list[-1] - energy_res_list[-2])/energy_res_list[-2]*100:.3f} %']

    for i in range(len(infos)):
        print(f'{infos[i]:<20}{row1[i]:<15}{row2[i]:<15}{row3[i]:<15}{row4[i]:<15}')
    


In [ ]:
calibrate_axis()
calibrate_resolution()
print_calibration_info()

## Peak positions fitting

In [ ]:
# these could be put in a separate file
# if so, x and s should be passed as arguments


def energy_index(energy):
    # given an energy in keV, return the index of that energy in the spectrum
    return np.abs(x - energy).argmin()


def energy_counts(energy):
    # given an energy in keV, return the counts at that energy
    energy_index = energy_index(energy)
    return s.data[energy_index]


def range_counts(start, stop):
    # given a range in keV, return the counts in that range
    return s.data[energy_index(start), energy_index(stop)].sum()


def max_counts(energy, buffer=10):
    # get the highest counts in +- 10 channels around a given energy
    i = np.argmin(np.abs(x - energy))
    return s.data[i-buffer:i+buffer].max()
     

def gaussian_counts(line):
    # give the sum of counts from a gaussian fit of a line
    return (m[line].function(x) * s.axes_manager[0].scale).sum()

def gaussian_max_counts(line):
    # give the max counts from a gaussian fit of a line
    return m[line].function(x).max() * s.axes_manager[0].scale


def bg_point(energy):
    # give the background counts at a certain energy
    return m[-1].function(energy) * s.axes_manager[0].scale


def bg_range(range_array):
    # give the background sum counts in range, where range is an array of energies
    return (m[-1].function(range_array) * s.axes_manager[0].scale).sum()

    
def kev_to_channels(energy):
    return np.argmin(np.abs(x - energy))


def theoretical_energy(line):
    # returns the theoretical energy of a given line, e.g. 'Ga_Ka'
    element = line.split('_')[0]
    line_name = line.split('_')[1]
    return hs.material.elements[element]['Atomic_properties']['Xray_lines'][line_name]['energy (keV)']


In [ ]:
def calibrate_lines(rounds=2):
    # calibrating the lines using HyperSpy
    # calibrate energy rounds times, and width once
    for i in range(rounds):
        print(f'Calibrating peak positions, round {i+1} of {rounds}')
        m.calibrate_xray_lines(calibrate='energy', xray_lines='all', kind='single') # use kind='multi' for better results?
        if i == 0:
            m.calibrate_xray_lines(calibrate='width', xray_lines='all', kind='single')
    # m.calibrate_xray_lines(calibrate='sub_weight', xray_lines='all', kind='single')  # not necessary?


def make_lines_info(all_lines=info_on_all_lines):
    # make lists with line info
    lines_info = {}
    for i in range(len(m) - 1): # last component is the background
        if (all_lines == True) or (m[i].name in lines_of_interest):
            lines_info[m[i].name] = {
                'true_energy': theoretical_energy(m[i].name),
                'calib_energy': m[i].centre.value,
                'area': gaussian_counts(i),
                'max_counts': gaussian_max_counts(i),
                'sigma': m[i].sigma.value,
                'fwhm': m[i].fwhm * 1000
            }
            # line_names = (m[i].name)
            # true_energy = (theoretical_energy(m[i].name))
            # calib_energy = (m[i].centre.value)
            # area = (gaussian_counts(i))
            # max_counts = (gaussian_max_counts(i))
            # sigma = (m[i].sigma.value)
            # fwhm = (m[i].fwhm) * 1000
            # lines_info.append([line_names, true_energy, calib_energy, area, max_counts, sigma, fwhm])
    return lines_info


def print_lines_info(give_return=False):
    # print line info
    headers = ['Line', 'True E [keV]', 'Calib. E [keV]', 'Area [counts]', 'Max (fit)', 'Sigma [keV]', 'FWHM [eV]']
    table = ''
    for i in range(len(headers)):
        table += f'{headers[i]:<15}'
    table += '\n'
    for line in lines_info:
    #   # this looks bad, but it is to align the columns and adjust the number of decimals
        one_line_info = f'{line:<15}'
        one_line_info += f'{lines_info[line]["true_energy"]:<15.4f}'
        one_line_info += f'{lines_info[line]["calib_energy"]:<15.4f}'
        one_line_info += f'{lines_info[line]["area"]:<15.1f}'
        one_line_info += f'{lines_info[line]["max_counts"]:<15.1f}'
        one_line_info += f'{lines_info[line]["sigma"]:<15.4f}'
        one_line_info += f'{lines_info[line]["fwhm"]:<15.4f}'
        table += one_line_info + '\n'
    
    if give_return == True:
        return table
    else:
        print(table)

        

In [ ]:
calibrate_lines()
lines_info = make_lines_info()
print_lines_info()

In [ ]:
## K to L ratios

def peak_ratio(line1, line2):
    # give the K to L ratio of a line, e.g. 'Ga_Ka' to 'Ga_La'
    return gaussian_counts(line1) / gaussian_counts(line2)

def calculate_k_to_l_ratios():
    # calculate the K to L ratios for k_to_l_ratio_elements
    k_to_l_ratios = {}
    for element in k_to_l_ratio_elements:
        k_to_l_ratios[element] =  peak_ratio(element+'_Ka', element+'_La')
    return k_to_l_ratios

k_to_l_ratios = calculate_k_to_l_ratios()


In [ ]:
## FWTM to FWHM

def fwtm_to_fwhm_line(line, plot_ratio=False):
    s_wo_bg = s.deepcopy()
    s_wo_bg.data = s.data - m[-1].function(x)*s.axes_manager[0].scale
    window = m[line].sigma.value * 4 # window around the line
    line_energy = m[line].centre.value
    s_line_window = s_wo_bg.isig[line_energy-window:line_energy+window]
    fwhm = s_line_window.estimate_peak_width(factor=0.5, parallel=False).data[0]
    fwtm = s_line_window.estimate_peak_width(factor=0.1, parallel=False).data[0]
    # WARNING:hyperspy._signals.signal1d:Parallel operation is not supported on Windows. Setting `parallel=False`

    # TODO: Put this plotting in a separate function?
    if plot_ratio:
        plt.plot(s_line_window.axes_manager[0].axis, s_line_window.data, label=f'Signal without bg', marker='o')
        plt.axvline(line_energy-window, color='k', linestyle='--', label='window')
        plt.axvline(line_energy+window, color='k', linestyle='--')
        plt.axvline(theoretical_energy(line), color='r', linestyle='--', label='theoretical line center')
        plt.axvline(m[line].centre.value, color='g', linestyle='--', label='calibrated center')
        plt.plot([line_energy-fwhm/2, line_energy+fwhm/2], [lines_info[line]['max_counts']/2, lines_info[line]['max_counts']/2], label='fwhm')
        plt.plot([line_energy-fwtm/2, line_energy+fwtm/2], [lines_info[line]['max_counts']/10, lines_info[line]['max_counts']/10], label='fwtm')
        plt.title(f'{line} \n FWHM: {fwhm*1000:.2f} eV, FWTM: {fwtm*1000:.2f} eV, ratio: {fwtm/fwhm:.3f}')
        plt.legend()
        plt.show()
    
    return {'fwhm': fwhm*1000, 'fwtm': fwtm*1000, 'fwtm/fwhm': fwtm/fwhm}


def fwtm_to_fwhm_all():
    fwtm_to_fwhm = {}
    for line in fwtm_to_fwhm_lines:
        fwtm_to_fwhm[line] = fwtm_to_fwhm_line(line, plot_ratio=False)

    return fwtm_to_fwhm
        

fwtm_to_fwhm = fwtm_to_fwhm_all()

In [ ]:
fwtm_to_fwhm

## Total counts and background counts

In [ ]:
total_counts = s.data.sum()
# total_counts_m = m.signal.data.sum() # does not work, m.signal is just a view of s
print(f'total_counts: {total_counts:.1f}')

# counts in the background
background_counts = bg_range(x).sum()
print(f'background_counts: {background_counts:.1f}')

# counts in the spectrum without background
total_counts_wo_bg = total_counts - background_counts
print(f'total_counts_wo_bg: {total_counts_wo_bg:.1f}')

# ratio of total counts to total counts in background
print(f'total_counts/background_counts: {total_counts/background_counts:.1f}')

# ratio of total counts to total counts in spectrum without background
print(f'total_counts/total_counts_wo_bg: {total_counts/total_counts_wo_bg:.1f}')

# Fiori background at a certain peak
#TODO

In [ ]:
plt.close('all')

In [ ]:
# setting the parameters which have temporary arrays

scale = scale_list[-1]
offset = offset_list[-1]
energy_resolution = energy_res_list[-1]

In [ ]:
def print_output(scale=scale, offset=offset, energy_resolution=energy_resolution, total_counts=total_counts, 
background_counts=background_counts, lines_info=lines_info, k_to_l_ratios=k_to_l_ratios, fwtm_to_fwhm=fwtm_to_fwhm):
    print(f"scale: {scale}")
    print(f"offset: {offset}")
    print(f"energy_resolution: {energy_resolution}")
    print(f"total_counts: {total_counts}")
    print(f"background_counts: {background_counts}")
    print(print_lines_info(give_return=True))
    print(f"k_to_l_ratios: {k_to_l_ratios}")
    print(f"fwtm_to_fwhm: {fwtm_to_fwhm}")

print_output()

In [ ]:
# put output in a dictionary

def output_dict(scale=scale, offset=offset, energy_resolution=energy_resolution, total_counts=total_counts,
background_counts=background_counts, lines_info=lines_info, k_to_l_ratios=k_to_l_ratios, fwtm_to_fwhm=fwtm_to_fwhm):
    output = {'scale': scale, 'offset': offset, 'energy_resolution': energy_resolution, 'total_counts': total_counts,
    'background_counts': background_counts, 'lines_info': lines_info, 'k_to_l_ratios': k_to_l_ratios, 'fwtm_to_fwhm': fwtm_to_fwhm}
    return output


output = output_dict()
output

In [ ]:
plt.plot(x, m[-1].function(x) * s.axes_manager[0].scale)
plt.plot(x, s.data)

plt.show()

In [ ]:
plt.close('all')

In [ ]:
# TODO: discuss with Ton this Duane-Hunt method


path = 'data/Mæhlum_2022-09-06_EDS-SEM-APREO'
file = 'GaAs_10kV.emsa'

s_dh = hs.load(path + '/' + file, signal_type='EDS_TEM')  # have to pretent it's a TEM signal
s_dh.add_elements(elements)
if zero_peak:
    s_dh = s_dh.isig[zero_peak_end_index:]
Vacc = s_dh.metadata.Acquisition_instrument.TEM.beam_energy
x_max = s_dh.axes_manager[0].high_value  # highest x-axis value in keV, used in Duane-Hunt
x = s_dh.axes_manager[0].axis  # x-axis in keV





# Duane-Hunt (where the the real E_0 is)
def calculate_duane_hunt(buffer_start=3, buffer_end=0.2):
    if Vacc > x_max:
        print(f'Vacc={Vacc} > x_max={x_max}, Duane-Hunt not possible')
        return np.nan
    else:
        # doing a fit from Vacc-3 keV till Vacc-0.2 keV
        ...


buffer_start=2
buffer_end=0.1

dh_start = Vacc-buffer_start
dh_end = Vacc-buffer_end
s_end = s_dh.isig[dh_start:dh_end] # slice with keV
m_end = s_end.create_model(auto_background=False)
m_end.add_polynomial_background(order=1)
m_end.fit()

x_s_end = s_dh.isig[dh_start-0.5:dh_end+0.5].axes_manager[0].axis

plt.plot(x, s_dh.data, label='spectrum', marker='o')

plt.plot(x_s_end, m_end[-1].function(x_s_end)* s_dh.axes_manager[0].scale, label='bg lin. fit')
plt.plot(s_end.axes_manager[0].axis, s_end.data, marker='o', label=f'points used for lin. fit bg, kV: [{dh_start:.2f}, {dh_end:.2f}]')
plt.axhline(0, color='k', linestyle='--')
plt.legend()
plt.show()


# set y range from -20 to 200
plt.ylim(-20, 350)
plt.xlim(7,11)

# locate where the background is closest to zero
bg_zero_index = np.argmin(np.abs(m_end[-1].function(x_s_end)* s_dh.axes_manager[0].scale))
bg_zero_index

print(f'bg_zero_index: {bg_zero_index}')
print(f'x_s_end[bg_zero_index]: {x_s_end[bg_zero_index]}')
# calculate_duane_hunt()